# <div align="center" style="color: #ff5733;">Sil Account level tagging based on Item Purchased</div>

# Logic for this is to select the data based on the Item purchased. If the Item Purchased in Mobile phone then it is a Mobile store

Created by : Dwaipayan Chakroborti<br>
Requestor: Anastasia -- Refer to her group chat with Heaven, Biswa, Me and Sreekanth<br>
Dated: 2024-09-30

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [4]:
sq = """     
SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  lmt.new_loan_type,
  lmt.flagDisbursement,
  lmt.disbursementDateTime,
  lmt.disbursedLoanAmount,
  lmt.downPaymentAmount,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
  `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf
ON
  UPPER(asf.PURPLE_KEY) = UPPER(lmt.purpleKey) --
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.flagDisbursement = 1
 and lmt.disbursementDateTime is not null
  ;
"""

In [5]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 2ebfbaa9-55aa-4878-8eb8-aaa7c7bb0687 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [6]:
df.to_csv(r"LoanLevelPurchasedITEM.csv", index = False)

# New query with the code part from Marek also included for better clarity

In [17]:
sq = """SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  substr(cast(date_trunc(lmt.decision_date, MONTH) as string),0,10) as mth,
  lmt.employmentstatus,
  case
  when EXTRACT(HOUR FROM lmt.termsAndConditionsSubmitDateTime)<18 then "<18"
  else "18+" end as application_hour,
  case
  when upper(lmt.purposeDescription) like "%MOBILE%" then "mobile"
  else "non-mobile" end as goods_type,
  case
  when c.store_type=1 then "appliance"
  else "mobile" end as pos_type,
  c.mer_name as retailer,
  lmt.new_loan_type,
  lmt.loantype,
  lmt.flagApproval as approved,
  lmt.flagDisbursement as disbursed,
  lmt.flagrejection as rejected,
  lmt.disbursedloanamount as volume_out,
  lmt.disbursedLoanTenur as tenor,
  lmt.disbursementDateTime,
  lmt.downPaymentAmount,
  case
    when lmt.loanRequestAmount<=10000 then "<10k"
    when lmt.loanRequestAmount<=15000 then "<15k"
    when lmt.loanRequestAmount<=20000 then "<20k"
    when lmt.loanRequestAmount<=25000 then "<25k"
    when lmt.loanRequestAmount<=30000 then "<30k"
    else "30k+" end as ca_category,
  case
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.195 then "<20%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.295 then "<30%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.35 then "<35%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.4 then "<40%"
  else "40+" end as DP_range,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
  `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf
ON
  UPPER(asf.PURPLE_KEY) = UPPER(lmt.purpleKey) --
inner join `dl_loans_db_raw.tdbk_merchant_refferal_mtb` c on lmt.purpleKey=c.mer_refferal_code
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.loantype="BNPL"
  ;"""

In [18]:
dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 5b6f58b6-7921-4ac6-b4f9-193915ac94d1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [19]:
dfd.shape

(295285, 35)

In [20]:
dfd.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'purpleKey',
       'merchantName', 'startApplyDateTime', 'applicationStatus',
       'decision_date', 'mth', 'employmentstatus', 'application_hour',
       'goods_type', 'pos_type', 'retailer', 'new_loan_type', 'loantype',
       'approved', 'disbursed', 'rejected', 'volume_out', 'tenor',
       'disbursementDateTime', 'downPaymentAmount', 'ca_category', 'DP_range',
       'loanCategoryId', 'createdDate', 'brandName', 'skuNo', 'price',
       'product_id', 'product_category', 'product_description', 'STORE_TAG',
       'product_downpayment', 'product_crifcode'],
      dtype='object')

In [21]:
dfd.head()

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,...,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
0,c2ad9768-d985-4c84-8ad6-de07025d30fa,None,M9ZK6IU,Hardware Sugar Makati,2022-12-09 22:13:03,EXEMPT,NaT,None,SALARIED,18+,...,2022-12-09 14:12:54+00:00,None,None,None,1725,None,"PC, Laptop or Game consoles",None,0.0,115
1,7adc6157-bf3c-46ca-8c44-46f4223685de,60828049420017,M24OR61,Home Along San Ildefonso,2024-08-29 16:42:22,ACTIVATED,2024-08-29 16:48:18,2024-08-01,SALARIED,<18,...,2024-08-29 08:42:17+00:00,Canon,CNn.G3010,9695.00,1725,None,"PC, Laptop or Game consoles",Appliances,0.0,115
2,356f9eb4-8798-4b40-9d2d-da38e15a6f9c,60824036430011,M45PJQZ,Home Along GMA,2024-02-18 11:08:23,ACTIVATED,2024-02-18 11:11:16,2024-02-01,REMITTANCE BENEFICIARY,<18,...,2024-02-18 03:08:17+00:00,CANON,CNN.G3010,9695.00,1725,None,"PC, Laptop or Game consoles",Appliances,0.0,115
3,d78e8e40-7720-4bb1-9bd1-a109915d97f5,60824862640016,MNK9LJZ,Willy And Sons Iriga,2024-04-28 13:44:55,ACTIVATED,2024-04-28 13:57:41,2024-04-01,SELF EMPLOYED,<18,...,2024-04-28 05:44:20+00:00,Asus,E510KA-BR582W,23900.00,1725,None,"PC, Laptop or Game consoles",Appliances,0.0,115
4,a2478328-3c94-447b-9632-b87c6e42d952,60821756640011,M7ZLTH1,Home Along Muzon,2023-08-06 13:35:06,ACTIVATED,2023-08-06 13:42:30,2023-08-01,SALARIED,<18,...,2023-08-06 05:35:01+00:00,Acer,A315-2040 R9Z,36999.00,1725,None,"PC, Laptop or Game consoles",Appliances,0.0,115


In [22]:
print(dfd.dtypes)

digitalLoanAccountId                 object
loanAccountNumber                    object
purpleKey                            object
merchantName                         object
startApplyDateTime           datetime64[us]
applicationStatus                    object
decision_date                datetime64[us]
mth                                  object
employmentstatus                     object
application_hour                     object
goods_type                           object
pos_type                             object
retailer                             object
new_loan_type                        object
loantype                             object
approved                              Int64
disbursed                             Int64
rejected                              Int64
volume_out                          float64
tenor                                 Int64
disbursementDateTime         datetime64[us]
downPaymentAmount                    object
ca_category                     

In [23]:
datetime_columns = ['startApplyDateTime', 'decision_date', 'disbursementDateTime', 'createdDate']

# Remove timezone information
for col in datetime_columns:
    dfd[col] = dfd[col].dt.tz_localize(None)
    


In [24]:
dfd.to_excel("Sil_DisbursedData_20240930.xlsx", index = False)